In [1]:
import pandas as pd 
import numpy as np
import os 

In [26]:
train_val = pd.read_csv("/home/users/shreshtha.singh/qxr_lungcancer/datasets/LN_csvs/train_val_original_nodule_level_labels.csv")
old_fake_data = pd.read_csv("/home/users/shreshtha.singh/qxr_lungcancer/datasets/LN_csvs/fake_nodule_level_labels.csv")
test_csv = pd.read_csv("/home/users/shreshtha.singh/qxr_lungcancer/datasets/LN_csvs/test_nodule_level_labels.csv")
train_neg  = pd.read_csv("/home/users/shreshtha.singh/qxr_lungcancer/datasets/LN_csvs/nod_neg_train_val_split/train_negs.csv")
val_neg  = pd.read_csv("/home/users/shreshtha.singh/qxr_lungcancer/datasets/LN_csvs/nod_neg_train_val_split/val_negs.csv")

In [10]:
old_fake_data.filename.iloc[20001], old_fake_data.nodule_id.iloc[20001]

('max.dev2.78491427.x.0', 1)

In [30]:
old_fake_data['real']  = old_fake_data['filename'].apply(lambda x: 0 if ('.x.' in x or '.y.' in x) else 1)
old_fake_data['real'].value_counts()

real
0    37698
Name: count, dtype: int64

In [29]:
train_val['real']  = train_val['filename'].apply(lambda x: 0 if  ('.x.' in x or '.y.' in x) else 1)
train_val['real'].value_counts()

real
1    60363
Name: count, dtype: int64

In [18]:
old_fake_data_ = old_fake_data[old_fake_data['real'] == 1 ]

In [4]:
train_val.filename.nunique(), len(train_val)

(37196, 60363)

In [5]:
old_fake_data.filename.nunique(), len(old_fake_data)

(27619, 37698)

In [6]:
train_data = pd.concat([train_val, old_fake_data])

In [7]:
train_data.columns

Index(['filename', 'nodule_id', 'x_min', 'x_max', 'y_min', 'y_max', 'contour',
       'contour_area', 'img_x', 'img_y', 'solitary', 'cancer', '10-30mm',
       'tinynodule', 'homogenous', 'regular_border', 'mass', 'calcified',
       'diffuse', 'inhomogenous', '5-10mm', 'large', 'more_than_30mm',
       'irregular_border', 'inhomogenous_remarks', 'inhomogenous_mask_missing',
       'homogenous_remarks', 'homogenous_mask_missing', 'nodule_mask_missing'],
      dtype='object')

In [8]:
train_data['nodule'] = 1 
train_data['normal']  = 0 
train_data = train_data[['filename','nodule_id', 'nodule' ,'normal', 'homogenous', 'inhomogenous',
       'solitary', 'diffuse', 'regular_border', 'irregular_border',
       'tinynodule', 'calcified', 'cancer', 'mass', 'large']]
train_data = train_data.dropna(subset=['filename', 'nodule_id'])
train_data = train_data.fillna(-100)


In [9]:
train_data.filename.nunique() , len(train_data)

(64731, 97977)

In [12]:

# Fill NaN values with -100
train_data.fillna(-100, inplace=True)

# Convert all columns (except 'filename' and 'nodule_id') to integers
columns_to_convert = train_data.columns.difference(['filename', 'nodule_id'])
train_data[columns_to_convert] = train_data[columns_to_convert].astype(int)

# Define a function to perform the custom OR operation
def custom_or(group):
    # Start with the first row's values as the initial result
    result = group.iloc[0].copy()
    for col in group.columns:
        if col not in ['filename', 'nodule_id']:
            # Perform bitwise OR across the column, excluding -100 values
            valid_values = group[col][group[col] != -100]
            if len(valid_values) > 0:
                result[col] = valid_values.iloc[0]
                for value in valid_values[1:]:
                    result[col] |= value
            else:
                result[col] = -100
    return result

# Group by 'filename' and apply the custom OR operation
grouped_data = train_data.groupby('filename').apply(custom_or)

# Drop the 'nodule_id' column
train_data_unique = grouped_data.drop(columns=['nodule_id']).reset_index(drop=True)

# Display the resulting DataFrame
train_data_unique


,filename,nodule,normal,homogenous,inhomogenous,solitary,diffuse,regular_border,irregular_border,tinynodule,calcified,cancer,mass,large
0,ca.50kstudies.unit10.25b2f5424b22868782407cf65...,1,0,0,0,1,0,0,1,0,0,0,0,0
1,ca.50kstudies.unit10.5ee91c92facb3657b284cf6e0...,1,0,0,0,1,0,1,0,0,1,0,0,0
2,ca.50kstudies.unit10.7a8f0166ed8effc9951b9b9d6...,1,0,0,0,1,0,1,0,1,1,0,0,0
3,ca.50kstudies.unit10.83b67ecc060b1f22e83a4b23f...,1,0,0,0,0,0,0,0,1,1,0,0,0
4,ca.50kstudies.unit10.9a9c80bfe51cdf5c169bc70df...,1,0,0,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64726,medanta.ffe34ed5-504df7ec-5398e2eb-230b7c43-1b...,1,0,1,0,1,0,1,0,0,0,1,0,1
64727,medanta.ffe4e473-5adfdf64-0233e3fd-38fd72e6-a4...,1,0,1,0,0,1,1,0,1,0,0,0,0
64728,medanta.fff13fdc-e35a7190-2ff77253-8307709d-27...,1,0,0,1,0,1,0,0,0,0,0,0,0
64729,medanta.fffe7995-25a07e4c-5528374e-d06f1ef4-f5...,1,0,0,1,0,1,0,0,0,0,0,0,0


In [28]:
combined_neg = pd.concat([train_neg, val_neg])

In [29]:
combined_neg  = combined_neg[['filename','nodule' ,'normal', 'homogenous', 'inhomogenous',
       'solitary', 'diffuse', 'regular_border', 'irregular_border',
       'tinynodule', 'calcified', 'cancer', 'mass', 'large']]

In [44]:
all_old_data = pd.concat([train_data_unique,combined_neg] )

In [45]:
all_old_data.reset_index(drop=True, inplace=True)


In [47]:
all_old_data

,filename,nodule,normal,homogenous,inhomogenous,solitary,diffuse,regular_border,irregular_border,tinynodule,calcified,cancer,mass,large
0,ca.50kstudies.unit10.25b2f5424b22868782407cf65...,1,0,0,0,1,0,0,1,0,0,0,0.0,0
1,ca.50kstudies.unit10.5ee91c92facb3657b284cf6e0...,1,0,0,0,1,0,1,0,0,1,0,0.0,0
2,ca.50kstudies.unit10.7a8f0166ed8effc9951b9b9d6...,1,0,0,0,1,0,1,0,1,1,0,0.0,0
3,ca.50kstudies.unit10.83b67ecc060b1f22e83a4b23f...,1,0,0,0,0,0,0,0,1,1,0,0.0,0
4,ca.50kstudies.unit10.9a9c80bfe51cdf5c169bc70df...,1,0,0,0,1,0,1,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894013,max.dev1.27274323,0,0,0,0,0,0,0,0,0,0,0,0.0,0
1894014,ca.phaseii.unit4.e4d2697cff5450f47f0d6533ee8f5...,0,0,0,0,0,0,0,0,0,0,0,0.0,0
1894015,dmims.1.2.840.113564.54.90.0.0.34.201402141648...,0,0,0,0,0,0,0,0,0,0,0,0.0,0
1894016,medall.1.2.840.10008.1.24355.1138923001.118425...,0,0,0,0,0,0,0,0,0,0,0,0.0,0


In [50]:
all_old_data.to_csv("/fast_data_e2e11/piyush/training/training_csvs/old_train_data_1.8M.csv",index= False)

### testing data

In [128]:
test_csv_all = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_08-08-24.csv")

In [110]:
internal_test_csv = test_csv[test_csv['src'] == 'internal testset' ]

In [111]:
test_csv_cleaned = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/cxr_data/filt_csvs/combined_all_test_tag_ex_31_07_24.csv")

In [112]:
test_csv_cleaned['src'] =  'internal testset' 
test_csv_cleaned.rename(columns= {'nipple_shadow' : 'nippleshadow'},inplace =True)

In [114]:
internal_test_csv.columns

Index(['filename', 'src', 'nodule', 'nippleshadow', 'calcification',
       'granuloma', 'mass', 'ribfracture', 'consolidation', 'fibrosis',
       'ct recommended', 'solitary', 'artifact', 'subtlety', 'nipple_marker',
       'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule',
       'tubr_nodule', 'LN', 'LNMS', 'bbox', 'png_path'],
      dtype='object')

In [115]:
len(internal_test_csv), len(test_csv_cleaned)

(317186, 319394)

In [123]:
test_csv_ = test_csv_cleaned.merge(internal_test_csv[['filename', 'LN', 'LNMS', 'bbox', 'png_path']], on= "filename", how ="left")

In [124]:
test_csv_ = test_csv_[['filename', 'src', 'nodule', 'nippleshadow', 'calcification',
       'granuloma', 'mass', 'ribfracture', 'consolidation', 'fibrosis',
       'ct recommended', 'solitary', 'artifact', 'subtlety', 'nipple_marker',
       'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule',
       'tubr_nodule', 'LN', 'LNMS', 'bbox', 'png_path']]

In [125]:
exclude_columns = ['filename', 'src', 'bbox', 'png_path']
columns_to_convert = test_csv_.columns.difference(exclude_columns)
test_csv_[columns_to_convert] = test_csv_[columns_to_convert].fillna(-100).astype(int)
test_csv_.dropna(inplace= True)

In [129]:
test_csv_all = test_csv_all[test_csv_all['src'] != 'internal testset']

In [130]:
len(test_csv_all)

218997

In [139]:
nodule_1_cases = test_csv_[test_csv_['nodule'] == 1]
remaining_needed = 50000 - len(nodule_1_cases)
remaining_cases = test_csv_[test_csv_['nodule'] != 1].sample(n=remaining_needed, random_state=42)
smaller_test_csv_ = pd.concat([nodule_1_cases, remaining_cases]).reset_index(drop=True)
smaller_test_csv_


,filename,src,nodule,nippleshadow,calcification,granuloma,mass,ribfracture,consolidation,fibrosis,...,nipple_marker,fibronodule,fibrolesion,miliarypattern,infectious_nodule,tubr_nodule,LN,LNMS,bbox,png_path
0,ims.file_045024115626417,internal testset,1,-100,1,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,1,-100,[],/models_common_e2e/cxr_data/testing/images/ims...
1,krsnaa.1.2.392.200036.9125.9.0.253373253.57308...,internal testset,1,-100,0,0,0,0,1,1,...,-100,-100,-100,0,-100,-100,1,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
2,krsnaa.1.2.392.200036.9125.4.0.504345363.10587...,internal testset,1,-100,0,0,0,0,1,0,...,-100,-100,-100,0,-100,-100,1,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
3,dmims.1.2.840.113564.54.192.168.101.16.2012121...,internal testset,1,-100,1,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,1,-100,[],/models_common_e2e/cxr_data/testing/images/dmi...
4,medall.1.3.6.1.4.1.25403.52241339060.1316.2017...,internal testset,1,-100,0,0,0,0,0,1,...,-100,-100,-100,0,-100,-100,1,-100,[],/models_common_e2e/cxr_data/testing/images/med...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,ca.50kstudies.unit4.b8e37531cac38a3b04180ea55f...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/ca....
49996,medall.1.2.392.200036.9125.2.19652107106227164...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/med...
49997,naw.1.3.51.0.7.1971468772.13907.18763.48657.35...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/naw...
49998,krsnaa.1.2.392.200036.9125.9.0.253560540.25678...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/krs...


In [133]:
test_csv_all_new = pd.concat([test_csv_all,test_csv_]).reset_index(drop=True)

In [140]:
test_csv_all_smaller = pd.concat([test_csv_all,smaller_test_csv_]).reset_index(drop=True)

In [148]:
test_csv_all_smaller[test_csv_all_smaller['bbox'] != '[]']['src'].value_counts()


src
fda          179
nlst_jsrt    154
Name: count, dtype: int64

In [149]:
test_csv_all_smaller.src.value_counts()

src
padchest            107709
segmed               66248
internal testset     50000
sbri                 31346
vrad                 11727
nlst_jsrt              821
mgh                    676
fda                    470
Name: count, dtype: int64

In [178]:

test_csv_all_smaller['filename'] = test_csv_all_smaller['png_path'].apply(lambda x: os.path.basename(x).replace(".png", ""))

In [197]:
test_csv_all_smaller['normal'] = -100

In [209]:
test_csv_all_smaller.rename(columns ={'niiple_shadow': "nipple_shadow"} ,inplace =True)

In [210]:
test_csv_all_smaller.to_csv("/fast_data_e2e11/qxr_ln_trainings/testing_shreshtha/combined_test_csv_2.5M_13-08-24.csv",index =False)

In [208]:
test_csv_all_smaller

,filename,src,nodule,niiple_shadow,calcification,granuloma,mass,ribfracture,consolidation,fibrosis,...,fibronodule,fibrolesion,miliarypattern,infectious_nodule,tubr_nodule,LN,LNMS,bbox,png_path,normal
0,1.2.392.200036.9125.4.0.2955215972.839066288.7...,sbri,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-100,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/1.2...,-100
1,f26315bb441a4aeaa6bb5bd6b45c6ad9,sbri,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-100,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/f26...,-100
2,02e743e928ca46909c38e185c854061f,sbri,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-100,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/02e...,-100
3,1c710196ab6e4254b7e31209ce156a01,sbri,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-100,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/1c7...,-100
4,1.2.392.200036.9125.4.0.1378179521.4093715120....,sbri,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-100,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/1.2...,-100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268992,ca.50kstudies.unit4.b8e37531cac38a3b04180ea55f...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/ca....,-100
268993,medall.1.2.392.200036.9125.2.19652107106227164...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/med...,-100
268994,naw.1.3.51.0.7.1971468772.13907.18763.48657.35...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/naw...,-100
268995,krsnaa.1.2.392.200036.9125.9.0.253560540.25678...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/krs...,-100


In [189]:
test_csv_all_new['filename'] = test_csv_all_new['png_path'].apply(lambda x: os.path.basename(x).replace(".png", ""))
test_csv_all_new.to_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_13-08-24.csv",index =False)

In [3]:
test_csv = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_13-08-24.csv")

In [22]:
total_ribfracture = len(test_csv[test_csv["ribfracture"] == 1 ]) 
total_mass = len(test_csv[test_csv["mass"] == 1 ])
total_nippleshadow = len(test_csv[test_csv["nippleshadow"] == 1 ])

print(total_ribfracture, total_mass,total_nippleshadow)

1925 1935 1223


In [6]:
test_csv.filename.nunique(), test_csv.png_path.nunique() ,len(test_csv)

(533345, 533349, 533350)

In [5]:
from tqdm import tqdm
import os

# List of filenames from the DataFrame
filenames = test_csv.filename.to_list()

# Adding tqdm to the loop for progress tracking
for filename in tqdm(filenames, desc="Checking Files"):
    path = "/fast_data_e2e_1/cxr/qxr_ln_data/testing_data_masks/mass/" + filename + '.png'
    if os.path.exists(path):
        continue
    else:
        print(filename)
        break


Checking Files:  25%|██▍       | 133336/533350 [01:27<04:21, 1527.99it/s]

pad.216840111366964012819207061112010243133749640_02-123-098


In [6]:
test_csv_ = test_csv[test_csv['filename'] == 'pad.216840111366964012819207061112010243133749640_02-123-098']

In [12]:
test_csv_ = test_csv[133336: 2*133336]

In [15]:
png_paths = test_csv.png_path.to_list()
for png_path in tqdm(png_paths, desc="Checking Files"):
    if os.path.exists(png_path):
        continue
    else:
        print(filename)
        break


Checking Files: 100%|██████████| 533350/533350 [02:42<00:00, 3277.70it/s]  


In [ ]:
test_csv